In [23]:
import math
import random
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [24]:
def isValidNumberString(s):
  if len(s) == 0:
    return False
  if  len(s) > 1 and s[0] == "-":
      s = s[1:]
  for c in s:
    if c not in "0123456789.":
      return False
  return True


In [25]:
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

In [26]:
def lineToTuple(line):
    # remove leading/trailing witespace and newlines
    cleanLine = line.strip()
    # get rid of quotes
    cleanLine = cleanLine.replace('"', '')
    # separate the fields
    lineList = cleanLine.split(",")
    # convert strings into numbers
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    return lineTuple

In [27]:
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset

In [28]:
def printTable(instances):
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print(line)

In [29]:
def extractAttribute(instances, index):
    result = []
    for instance in instances:
        result.append(instance[index])
    return result


In [30]:
def euclidean_distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    distance = 0
    for i in range(1, len(instance1)):
        distance += (instance1[i] - instance2[i])**2
    return math.sqrt(distance)

In [31]:
def cosine_distance(instance1, instance2):
  if instance1 == None or instance2 == None:
        return float("inf")
  A = np.array(instance1[1:],dtype=float)
  B = np.array(instance2[1:],dtype=float)
  distance = 1 - np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))
  return distance

In [32]:
def jaccard(instance1, instance2):
  if instance1 == None or instance2 == None:
        return float("inf")
  A = np.array(instance1[1:],dtype=float)
  B = np.array(instance2[1:],dtype=float)
  return 1 - (np.sum(np.minimum(A,B), axis = 0)/np.sum(np.maximum(A, B), axis = 0)) 

In [33]:
def calculateSSE(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(1, len(instance1)):
        sumOfSquares += (instance1[i] - instance2[i])**2
    return sumOfSquares

In [34]:
def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes-1)
    for instance in instanceList:
        for i in range(1, numAttributes):
            means[i] += instance[i]
    for i in range(1, numAttributes):
        means[i] /= float(numInstances)
    return tuple(means)

In [35]:
def assign(instance, centroids, dist_func):
  if(dist_func == "Euclidean"):
    minDistance = euclidean_distance(instance, centroids[0])
  elif(dist_func == "Cosine"):
    minDistance = cosine_distance(instance, centroids[0])
  else:
    minDistance = jaccard(instance, centroids[0])

  minDistanceIndex = 0
  for i in range(1, len(centroids)):
      if(dist_func == "Euclidean"):
        d = euclidean_distance(instance, centroids[i])
      elif(dist_func == "Cosine"):
        d = cosine_distance(instance, centroids[i])
      else:
        d = jaccard(instance, centroids[i])
      if (d < minDistance):
          minDistance = d
          minDistanceIndex = i
  return minDistanceIndex

In [36]:
def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

In [37]:
def assignAll(instances, centroids, dist_func):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = assign(instance, centroids, dist_func)
        clusters[clusterIndex].append(instance)
    return clusters

In [38]:
def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "centroid" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

In [39]:

def kmeans(instances, k, dist_func,initCentroids=None):

    result = {}
    sse_list=[]
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []

    iteration = 0
    start = time.time()
    while (centroids != prevCentroids):
        #if(iteration > 50):
        #  break
        iteration += 1
        clusters = assignAll(instances, centroids,dist_func)

        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids)

        sse_list.append(withinss)

    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["SSE"] = sse_list
    result["iterations"] = iteration
   # print("Time taken:", time.time() - start)
    return result

In [40]:
def computeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += calculateSSE(centroid, instance)
    return result


In [41]:
dataset = loadCSV("data.csv")
print(f'===== Euclidean =====')
euclidean_clustering = kmeans(dataset, 10,"Euclidean")
print(f'List of SSE: ',euclidean_clustering["SSE"])

===== Euclidean =====
List of SSE:  [27917074121.523205, 26775873976.08951, 26265015528.52319, 25979038131.553997, 25850645430.822266, 25770255880.964504, 25701932869.85813, 25623542191.39995, 25560387219.488422, 25531493104.001617, 25514741417.28083, 25499098059.453377, 25482269990.413998, 25465469564.287975, 25449110946.886578, 25434649152.81702, 25422267133.615936, 25412818608.070465, 25406129465.5411, 25401383131.042393, 25398535164.476086, 25396073802.111225, 25393798811.377357, 25392595641.247116, 25391040218.34923, 25389954702.250835, 25388909878.922306, 25387951598.368145, 25386927786.39087, 25385965983.275604, 25385454364.964687, 25384999245.573887, 25384593114.09205, 25384183563.10846, 25383952569.981426, 25383738375.927803, 25383576500.342953, 25383438417.252323, 25383379182.066257, 25383342272.88517, 25383325605.334656, 25383286740.534073, 25383253560.248787, 25383229986.514698, 25383204947.26184, 25383204947.26184]


In [42]:
print(f'===== Cosine =====')
cosine_clustering = kmeans(dataset, 10,"Cosine")
print(f'List of SSE: ',cosine_clustering["SSE"])

===== Cosine =====
List of SSE:  [29136224412.692673, 26871763660.673187, 26351380958.342552, 26181854745.73397, 26059101483.409496, 25970021330.313644, 25891379595.750336, 25820336036.47461, 25770278972.512936, 25733744214.448948, 25707155657.81296, 25688130692.85402, 25675848570.03582, 25667776966.47856, 25659092504.37076, 25654947071.651066, 25645559126.14778, 25633048463.306816, 25615784773.89095, 25598008418.63894, 25578570388.77308, 25549755382.654945, 25530720694.81366, 25524934704.93324, 25522150176.73685, 25521860668.479748, 25518773997.087467, 25518870399.79128, 25519484567.667664, 25517735801.990555, 25516937478.88886, 25517677980.821846, 25516377732.449482, 25515869576.165253, 25515141927.450493, 25515526661.725018, 25514161058.660664, 25513083551.41818, 25511433488.790176, 25511863325.41518, 25510241020.660084, 25508968371.02589, 25508729177.704426, 25507487287.157978, 25506824520.6877, 25506958959.940895, 25505683488.355408, 25504627820.32511, 25503355177.04529, 255020309

In [43]:

print(f'===== Jaccard =====')
jaccard_clustering = kmeans(dataset, 10,"Jaccard")
print(f'List of SSE: ',jaccard_clustering["SSE"])

===== Jaccard =====
List of SSE:  [27903727083.39925, 26409655191.375572, 25947225163.345352, 25802783787.794018, 25759817773.6302, 25737123204.896515, 25719498310.37204, 25707134715.750908, 25698299813.328495, 25687045302.996506, 25667509839.356045, 25646942734.830822, 25636147898.68032, 25628364568.494278, 25623687902.502842, 25623171972.103832, 25622070904.22468, 25619735383.549488, 25617307051.20753, 25615341422.17658, 25616211163.527344, 25615857959.747677, 25614526766.593227, 25613093389.221085, 25610002083.518684, 25607436038.255505, 25603821740.928665, 25599327850.724937, 25592715510.462883, 25585206048.773163, 25575038939.906475, 25561983056.034, 25539540902.70724, 25504919942.297558, 25466025285.82878, 25440442951.82105, 25424378159.944336, 25417821851.937916, 25416992929.652317, 25415092802.37866, 25413314267.98081, 25412542234.878025, 25413453855.22482, 25412860463.42128, 25413135868.385666, 25414007190.182625, 25414303097.248055, 25414552098.27378, 25414524401.16253, 25414